In [1]:
import pandas as pd
import glob

In [2]:
#Данные представленные в таблице имеют лишь колонку с датой и количеством. 
#Необходимо создать новый список который будет содержать код определяющий страну и вид энергии.
#В этом поможет файл "BP_metadata.csv".
#Из него нам потребуется только колонка с кодом и колонка с описанием.


In [3]:
df_meta = pd.read_csv('data/BP_metadata.csv', usecols = ['code', 'description'])
df_meta.head()

,code,description
0,BIOFUEL_PROD_D_ARG,Biofuel Production - Daily Average - Argentina...
1,BIOFUEL_PROD_D_AUS,Biofuel Production - Daily Average - Australia...
2,BIOFUEL_PROD_D_AUT,Biofuel Production - Daily Average - Austria. ...
3,BIOFUEL_PROD_D_BEL,Biofuel Production - Daily Average - Belgium. ...
4,BIOFUEL_PROD_D_BRA,Biofuel Production - Daily Average - Brazil. 1...


In [4]:
#Выведем строки в описании которых присутствует "Russian" и создадим соответствующий список и сохраним в новый файл
meta_rus = df_meta[df_meta['description'].str.contains('Russian')]
meta_rus.head()

,code,description
141,C02_EMMISSIONS_RUS,Carbon Dioxide (CO2) Emmissions - Russian Fede...
234,COAL_CONSUM_O_RUS,Coal Consumption - Oil Equivalent - Russian Fe...
317,COAL_PROD_O_RUS,Coal Production - Oil Equivalent - Russian Fed...
337,COAL_PROD_RUS,Coal Production - Russian Federation. Million ...
425,ELEC_GEN_RUS,Electricity Generation - Russian Federation. T...


In [5]:
meta_rus.to_csv('data/meta/meta_rus.csv', index=False)

In [6]:
#Аналогично сделаем для стран Китай и Германия.
meta_chi = df_meta[df_meta['description'].str.contains('China')]
meta_chi.head()

,code,description
7,BIOFUEL_PROD_D_CHN,Biofuel Production - Daily Average - China. 10...
42,BIOFUEL_PROD_O_CHN,Biofuel Production - Oil Equivalent - China. T...
84,C02_EMMISSIONS_CHN,Carbon Dioxide (CO2) Emmissions - China. Milli...
100,C02_EMMISSIONS_HKG,Carbon Dioxide (CO2) Emmissions - China Hong K...
177,COAL_CONSUM_O_CHN,Coal Consumption - Oil Equivalent - China. Mil...


In [7]:
meta_chi.to_csv('data/meta/meta_chi.csv', index=False)

In [8]:
meta_ger = df_meta[df_meta['description'].str.contains('Germany')]
meta_ger.head()

,code,description
9,BIOFUEL_PROD_D_DEU,Biofuel Production - Daily Average - Germany. ...
44,BIOFUEL_PROD_O_DEU,Biofuel Production - Oil Equivalent - Germany....
88,C02_EMMISSIONS_DEU,Carbon Dioxide (CO2) Emmissions - Germany. Mil...
181,COAL_CONSUM_O_DEU,Coal Consumption - Oil Equivalent - Germany. M...
265,COAL_PROD_DEU,Coal Production - Germany. Million Tonnes. Com...


In [9]:
meta_ger.to_csv('data/meta/meta_ger.csv', index=False)

In [10]:
#Далее нам нужно объеденить таблицы, содержащие информацию по странам. Чтобы по каждой стране был отдельной DataFrame.
# Создадим DataFrame и изменим колонку "Value" на код энергии.

In [11]:
df_C02_EMMISSIONS_RUS = pd.read_csv('data/RUS/BP-C02_EMMISSIONS_RUS.csv').rename(columns={"Date": "Date", "Value": "C02_EMMISSIONS_RUS"})
df_C02_EMMISSIONS_RUS.head()

,Date,C02_EMMISSIONS_RUS
0,2020-12-31,1482.188474
1,2019-12-31,1595.685634
2,2018-12-31,1605.961797
3,2017-12-31,1548.600760
4,2016-12-31,1566.991267


In [12]:
#Этот процесс проделаем в цикле

In [13]:
path = 'data/RUS'
list_name = glob.glob(path + "/*.csv")
for i in range(len(list_name)):
    list_name[i] = list_name[i].replace('data/RUS\\BP-', '')
    list_name[i] = list_name[i].replace('.csv', '')
ls = []
for i in range(len(list_name)):
    ls.append(pd.DataFrame(pd.read_csv('data/RUS/BP-' + list_name[i] + '.csv').rename(columns={"Date": "Date", "Value": list_name[i]})))    


In [14]:
ls[0].head()

,Date,C02_EMMISSIONS_RUS
0,2020-12-31,1482.188474
1,2019-12-31,1595.685634
2,2018-12-31,1605.961797
3,2017-12-31,1548.600760
4,2016-12-31,1566.991267


In [15]:
#Теперь у нас есть список со всеми данными для России. Объеденим их в один DataFrame

In [16]:
df_RUS = pd.DataFrame({"Date": []})
for i in range(len(ls)):
    df_RUS = pd.merge(left=df_RUS, right=ls[i], on='Date', how='outer')
df_RUS.index = df_RUS.Date
del df_RUS['Date']

In [27]:
df_RUS.head()

,C02_EMMISSIONS_RUS,COAL_CONSUM_O_RUS,COAL_PROD_O_RUS,COAL_PROD_RUS,ELEC_GEN_RUS,GAS_CONSUM_D_RUS,GAS_CONSUM_O_RUS,GAS_CONSUM_RUS,GAS_PROD_D_RUS,GAS_PROD_O_RUS,...,OIL_PROD_RUS,OIL_REF_CAP_RUS,OIL_RESERVES_RUS,PRIMARY_CONSUM_O_RUS,SOLAR_CAP_RUS,SOLAR_CONSUM_O_RUS,SOLAR_CONSUM_RUS,WIND_CAP_RUS,WIND_CONSUM_O_RUS,WIND_CONSUM_RUS
Date,,,,,,,,,,,,,,,,,,,,,
2020-12-31,1482.188474,NaN,NaN,399.664238,1085.418730,40.861130,NaN,423.483407,61.491342,NaN,...,524.404217,6736.0,107.804239,28.880287,1427.82,NaN,NaN,945.3,NaN,NaN
2019-12-31,1595.685634,NaN,NaN,440.905783,1118.143230,42.988367,NaN,444.312721,65.699286,NaN,...,573.388845,6676.0,107.804239,30.023607,1276.32,NaN,NaN,101.9,NaN,NaN
2018-12-31,1605.961797,88.031641,220.15432,441.576471,1109.197790,43.973945,NaN,454.499313,64.738353,NaN,...,567.884888,6551.0,107.220779,30.243229,535.32,0.129217,NaN,51.9,0.051532,NaN
2017-12-31,1548.600760,83.930000,205.80000,412.540341,1091.184000,41.710060,NaN,431.100591,61.492102,NaN,...,558.546697,6545.0,106.591534,29.168648,225.32,0.120899,0.5440,10.9,0.031735,0.1390
2016-12-31,1566.991267,89.250000,193.97000,386.613471,1090.970224,40.587391,NaN,420.646390,56.858089,NaN,...,558.479079,6543.0,106.282890,28.993425,76.32,0.105965,0.4683,10.9,0.032176,0.1422


In [19]:
#Проделаем то же самое с Китаем и Германией

In [28]:
path = 'data/GER'
list_name = glob.glob(path + "/*.csv")
for i in range(len(list_name)):
    list_name[i] = list_name[i].replace('data/GER\\BP-', '')
    list_name[i] = list_name[i].replace('.csv', '')
ls = []
for i in range(len(list_name)):
    ls.append(pd.DataFrame(pd.read_csv('data/GER/BP-' + list_name[i] + '.csv').rename(columns={"Date": "Date", "Value": list_name[i]})))  

df_GER = pd.DataFrame({"Date": []})
for i in range(len(ls)):
    df_GER = pd.merge(left=df_GER, right=ls[i], on='Date', how='outer')

df_GER.index = df_GER.Date
del df_GER['Date']
df_GER.head()

,BIOFUEL_PROD_D_DEU,BIOFUEL_PROD_O_DEU,C02_EMMISSIONS_DEU,COAL_CONSUM_O_DEU,COAL_PROD_DEU,COAL_PROD_O_DEU,ELEC_GEN_DEU,GAS_CONSUM_DEU,GAS_CONSUM_D_DEU,GAS_CONSUM_O_DEU,...,OIL_CONSUM_DEU,OIL_CONSUM_D_DEU,OIL_REF_CAP_DEU,PRIMARY_CONSUM_O_DEU,SOLAR_CAP_DEU,SOLAR_CONSUM_DEU,SOLAR_CONSUM_O_DEU,WIND_CAP_DEU,WIND_CONSUM_DEU,WIND_CONSUM_O_DEU
Date,,,,,,,,,,,,,,,,,,,,,
2021-12-31,54.258232,NaN,NaN,NaN,126.000,NaN,584.525694,90.530311,8.759034,NaN,...,95.491599,2045.180180,2120.619205,12.641714,58459.0,NaN,NaN,63760.0,NaN,NaN
2020-12-31,62.968571,NaN,604.875456,NaN,107.400,NaN,573.641487,87.112500,8.405324,NaN,...,96.482385,2049.323925,2061.537397,12.356541,53719.0,NaN,NaN,62188.0,NaN,NaN
2019-12-31,65.753713,NaN,681.482545,NaN,131.300,NaN,607.044227,89.283028,8.638356,NaN,...,106.550640,2269.879508,2062.441096,13.302727,48912.0,NaN,NaN,60742.0,NaN,NaN
2018-12-31,62.955768,3444.890556,734.490511,66.399159,168.842,37.590493,640.175188,85.850722,8.306272,NaN,...,106.001246,2254.685577,2061.537397,13.654258,45156.0,NaN,10.445762,58721.0,NaN,25.250034
2017-12-31,61.276140,3293.476163,760.978787,71.533128,175.122,39.419130,651.417259,87.738472,8.488917,NaN,...,111.304790,2374.479957,2068.867397,14.005308,42291.0,39.895,8.915464,55580.0,106.614,23.915690


In [29]:
path = 'data/CHI'
list_name = glob.glob(path + "/*.csv")
for i in range(len(list_name)):
    list_name[i] = list_name[i].replace('data/CHI\\BP-', '')
    list_name[i] = list_name[i].replace('.csv', '')
ls = []
for i in range(len(list_name)):
    ls.append(pd.DataFrame(pd.read_csv('data/CHI/BP-' + list_name[i] + '.csv').rename(columns={"Date": "Date", "Value": list_name[i]})))

df_CHI = pd.DataFrame({"Date": []})
for i in range(len(ls)):
    df_CHI = pd.merge(left=df_CHI, right=ls[i], on='Date', how='outer')

df_CHI.index = df_CHI.Date
del df_CHI['Date']
df_CHI.head()

,BIOFUEL_PROD_D_CHN,BIOFUEL_PROD_O_CHN,C02_EMMISSIONS_CHN,COAL_CONSUM_O_CHN,COAL_PROD_CHN,COAL_PROD_O_CHN,ELEC_GEN_CHN,GAS_CONSUM_CHN,GAS_CONSUM_D_CHN,GAS_CONSUM_O_CHN,...,OIL_REF_CAP_CHN,OIL_REF_THR_CHN,OIL_RESERVES_CHN,PRIMARY_CONSUM_O_CHN,SOLAR_CAP_CHN,SOLAR_CONSUM_CHN,SOLAR_CONSUM_O_CHN,WIND_CAP_CHN,WIND_CONSUM_CHN,WIND_CONSUM_O_CHN
Date,,,,,,,,,,,,,,,,,,,,,
2021-12-31,63.899017,NaN,NaN,NaN,4126.0000,NaN,8534.250,378.694414,36.639631,NaN,...,16990.260063,NaN,NaN,157.647225,306402.800,NaN,NaN,328973.430,NaN,NaN
2020-12-31,57.520908,NaN,9899.334721,NaN,3901.5771,NaN,7779.060,336.617257,32.479576,NaN,...,16691.414110,NaN,25.962757,147.580473,253417.800,NaN,NaN,282112.670,NaN,NaN
2019-12-31,54.669214,NaN,9810.456419,NaN,3846.3320,NaN,7503.428,308.376844,29.836231,NaN,...,16199.199589,NaN,25.962757,143.922566,204574.800,NaN,NaN,209581.910,NaN,NaN
2018-12-31,45.152767,3099.378410,9652.687227,1906.725000,3697.7358,1828.827,7166.133,283.926008,27.470551,NaN,...,15655.474247,NaN,26.190090,138.882849,175015.864,NaN,40.163823,184664.877,NaN,82.816672
2017-12-31,46.817161,2146.587912,9466.360373,1890.425612,3523.5618,1746.612,6604.447,241.252186,23.341752,NaN,...,15230.986918,NaN,25.948200,133.603714,130801.290,108.179306,26.655202,164374.477,286.080694,66.751143


In [22]:
#Сохраним результаты. На этом первая часть подготовки данных для анализа заканчивается

In [30]:
df_RUS.to_csv('data/results/RUS.csv')
df_GER.to_csv('data/results/GER.csv')
df_CHI.to_csv('data/results/CHI.csv')